In [2]:
import pandas as pd

# Define the file path to your downloaded CSV file.
# Make sure the path is correct for your system.
file_path = 'C:/Users/USER/data analysis projects/stores_sales_forecasting.csv'

# Try loading the dataset with common encodings
try:
    df = pd.read_csv(file_path, encoding='utf-8')
    print("✅ Dataset loaded successfully with utf-8 encoding!")
except UnicodeDecodeError:
    print("❌ UnicodeDecodeError: UTF-8 failed. Trying 'latin-1' encoding...")
    try:
        df = pd.read_csv(file_path, encoding='latin-1') # Often works for these errors
        print("✅ Dataset loaded successfully with latin-1 encoding!")
    except UnicodeDecodeError:
        print("❌ UnicodeDecodeError: Latin-1 failed. Trying 'ISO-8859-1' encoding...")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1') # Also very common
            print("✅ Dataset loaded successfully with ISO-8859-1 encoding!")
        except UnicodeDecodeError:
            print("❌ UnicodeDecodeError: ISO-8859-1 failed. Trying 'cp1252' encoding...")
            try:
                df = pd.read_csv(file_path, encoding='cp1252') # Another common Windows encoding
                print("✅ Dataset loaded successfully with cp1252 encoding!")
            except Exception as e:
                print(f"❌ Error: Could not decode the file with common encodings. Original error: {e}")
                exit()
except FileNotFoundError:
    print(f"❌ Error: File not found at {file_path}. Please double-check the 'file_path' variable.")
    exit()
except Exception as e: # Catch any other unexpected errors during initial load
    print(f"❌ An unexpected error occurred during file loading: {e}")
    exit()

# If the file loaded successfully, print the first 5 rows and info to confirm
if 'df' in locals(): # Check if df variable exists after successful load
    print("\nFirst 5 rows of the dataset:")
    print(df.head())

    print("\nDataset Information:")
    df.info()

    print("\nDescriptive Statistics:")
    print(df.describe())

❌ UnicodeDecodeError: UTF-8 failed. Trying 'latin-1' encoding...
✅ Dataset loaded successfully with latin-1 encoding!

First 5 rows of the dataset:
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
3       6  CA-2014-115812    6/9/2014   6/14/2014  Standard Class    BH-11710   
4      11  CA-2014-115812    6/9/2014   6/14/2014  Standard Class    BH-11710   

     Customer Name   Segment        Country             City  ... Postal Code  \
0      Claire Gute  Consumer  United States        Henderson  ...       42420   
1      Claire Gute  Consumer  United States        Henderson  ...       42420   
2   Sean O'Donnell  Consumer  United States  Fort Lauderdale  ...       33311   
3  Brosina Hoffman  Consumer  United Stat

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define the file path to your NEW downloaded CSV file (e.g., 'US Superstore Data.csv').
# Make sure the path is correct for your system.
# IMPORTANT: Replace 'train.csv' with the correct filename for the dataset you download
# that *includes* the 'Profit' column.
file_path = 'C:/Users/USER/data analysis projects/US Superstore Data.csv' # <--- CHANGE THIS FILENAME

# Load the dataset into a pandas DataFrame.
try:
    df = pd.read_csv(file_path, encoding='ISO-8859-1') # Often, Superstore data uses this encoding
    print("✅ Dataset loaded successfully!")
    print("\nFirst 5 rows of the dataset after loading the correct file:")
    print(df.head())
    print("\nDataset Information after loading the correct file:")
    df.info()
except FileNotFoundError:
    print(f"❌ Error: File not found at {file_path}. Please double-check the 'file_path' variable and ensure you've downloaded the correct dataset with a 'Profit' column.")
    exit()

print("Starting EDA phase...")

# --- Data Cleaning & Wrangling (revisiting based on df.info() and df.describe()) ---

# Check for missing values
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Handle missing 'Postal Code' values. Median is a reasonable choice for numerical data.
# This ensures operations like geographical analysis don't break.
# The 'US Superstore Data.csv' often has missing 'Postal Code' in some non-US rows.
if 'Postal Code' in df.columns and df['Postal Code'].isnull().any():
    df['Postal Code'].fillna(df['Postal Code'].median(), inplace=True)
    print("\nFilled missing 'Postal Code' values with the median.")
# Convert Postal Code to int if it's float and has no NaNs now
if 'Postal Code' in df.columns and df['Postal Code'].dtype == 'float64' and df['Postal Code'].isnull().sum() == 0:
    df['Postal Code'] = df['Postal Code'].astype(int)


# Correcting Data Types for 'Order Date' and 'Ship Date'
# Common formats are 'DD/MM/YYYY' or 'MM/DD/YYYY'.
# The 'US Superstore Data.csv' often uses 'MM/DD/YYYY' so `infer_datetime_format=True` or `format='%m/%d/%Y'` works well.
# Let's try `infer_datetime_format=True` first, it's often robust.
# If that fails, go back to `dayfirst=True` or explicit `format`.
try:
    df['Order Date'] = pd.to_datetime(df['Order Date'], infer_datetime_format=True)
    df['Ship Date'] = pd.to_datetime(df['Ship Date'], infer_datetime_format=True)
except Exception as e:
    print(f"Could not infer date format, trying with dayfirst=True. Error: {e}")
    try:
        df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True)
        df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True)
    except Exception as e:
        print(f"Could not convert dates even with dayfirst=True. Please inspect date format. Error: {e}")
        # Fallback for specific known formats if the above fails
        # df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
        # df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y')


print("\nData types after conversion:")
print(df.info())

# Feature Engineering (creating new features from existing ones)
# Extract Year, Month, Day from Order Date for time-series analysis
df['Order Year'] = df['Order Date'].dt.year
df['Order Month'] = df['Order Date'].dt.month
df['Order Day'] = df['Order Date'].dt.day
df['Order DayOfWeek'] = df['Order Date'].dt.dayofweek # Monday=0, Sunday=6
df['Order WeekOfYear'] = df['Order Date'].dt.isocalendar().week.astype(int)

# Calculate 'Days to Ship'
df['DaysToShip'] = (df['Ship Date'] - df['Order Date']).dt.days

# Check for duplicates
print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")
if df.duplicated().sum() > 0:
    df.drop_duplicates(inplace=True)
    print("Duplicate rows removed.")

# --- NOW, THE REST OF THE EDA VISUALIZATIONS WILL WORK AS 'Profit' COLUMN SHOULD BE PRESENT ---

# 1. Sales and Profit Distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['Sales'], bins=50, kde=True)
plt.title('Distribution of Sales')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.yscale('log') # Sales often have a long tail, log scale helps visualize

plt.subplot(1, 2, 2)
sns.histplot(df['Profit'], bins=50, kde=True) # This line will now work
plt.title('Distribution of Profit')
plt.xlabel('Profit')
plt.ylabel('Frequency')
plt.yscale('log')
plt.tight_layout()
plt.show()

# Observe: Sales and Profit distributions are often skewed. Many small transactions, few large.
# Negative profits indicate losses.

# 2. Relationship between Sales and Profit
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Sales', y='Profit', data=df, alpha=0.6)
plt.title('Sales vs. Profit')
plt.xlabel('Sales')
plt.ylabel('Profit')
plt.xscale('log') # Log scale helps spread out points for better visibility
plt.show()

# Observe: General positive correlation, but many points with high sales and low/negative profit.
# This indicates products with high sales but poor profitability.

# 3. Profit Ratio
# Ensure Sales is not zero to avoid division by zero
df['Profit Ratio'] = (df['Profit'] / df['Sales']) * 100
# Handle division by zero if Sales is 0, setting Profit Ratio to 0 in such cases
df.loc[df['Sales'] == 0, 'Profit Ratio'] = 0
# Replace infinite values (if Sales was 0 and Profit was non-zero) with 0
df['Profit Ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)


plt.figure(figsize=(10, 6))
sns.histplot(df['Profit Ratio'], bins=50, kde=True)
plt.title('Distribution of Profit Ratio (%)')
plt.xlabel('Profit Ratio (%)')
plt.ylabel('Frequency')
plt.show()

# Observe: What's the typical profit margin? Are there many loss-making items?

# 4. Sales and Profit by Category and Sub-Category
fig, axes = plt.subplots(2, 1, figsize=(12, 16))

# Sales by Category
sales_by_category = df.groupby('Category')['Sales'].sum().sort_values(ascending=False)
sns.barplot(x=sales_by_category.index, y=sales_by_category.values, ax=axes[0], palette='viridis')
axes[0].set_title('Total Sales by Category')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Total Sales')
axes[0].ticklabel_format(style='plain', axis='y') # Prevent scientific notation

# Profit by Category
profit_by_category = df.groupby('Category')['Profit'].sum().sort_values(ascending=False)
sns.barplot(x=profit_by_category.index, y=profit_by_category.values, ax=axes[1], palette='magma')
axes[1].set_title('Total Profit by Category')
axes[1].set_xlabel('Category')
axes[1].set_ylabel('Total Profit')
axes[1].ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.show()

# Observe: Which categories are top performers in sales and profit? Are they consistent?

# Sales and Profit by Sub-Category (Top N)
plt.figure(figsize=(14, 8))
top_subcategories_sales = df.groupby('Sub-Category')['Sales'].sum().nlargest(15).index
sns.barplot(x='Sales', y='Sub-Category', data=df[df['Sub-Category'].isin(top_subcategories_sales)],
            estimator=sum, palette='viridis', ci=None, order=top_subcategories_sales)
plt.title('Top 15 Sub-Categories by Total Sales')
plt.xlabel('Total Sales')
plt.ylabel('Sub-Category')
plt.ticklabel_format(style='plain', axis='x')
plt.show()


plt.figure(figsize=(14, 8))
top_subcategories_profit = df.groupby('Sub-Category')['Profit'].sum().nlargest(15).index
sns.barplot(x='Profit', y='Sub-Category', data=df[df['Sub-Category'].isin(top_subcategories_profit)],
            estimator=sum, palette='magma', ci=None, order=top_subcategories_profit)
plt.title('Top 15 Sub-Categories by Total Profit')
plt.xlabel('Total Profit')
plt.ylabel('Sub-Category')
plt.ticklabel_format(style='plain', axis='x')
plt.show()

# Identify loss-making sub-categories
loss_making_subcategories = df.groupby('Sub-Category')['Profit'].sum().sort_values(ascending=True)
print("\nSub-Categories with Negative Profit:")
print(loss_making_subcategories[loss_making_subcategories < 0])

# Observe: Are the top sales sub-categories also the top profit sub-categories?
# Which sub-categories are consistently losing money? This is crucial for inventory/marketing.

# 5. Sales and Profit by Region/State/City
fig, axes = plt.subplots(2, 1, figsize=(12, 16))

# Sales by Region
sales_by_region = df.groupby('Region')['Sales'].sum().sort_values(ascending=False)
sns.barplot(x=sales_by_region.index, y=sales_by_region.values, ax=axes[0], palette='crest')
axes[0].set_title('Total Sales by Region')
axes[0].set_xlabel('Region')
axes[0].set_ylabel('Total Sales')
axes[0].ticklabel_format(style='plain', axis='y')

# Profit by Region
profit_by_region = df.groupby('Region')['Profit'].sum().sort_values(ascending=False)
sns.barplot(x=profit_by_region.index, y=profit_by_region.values, ax=axes[1], palette='flare')
axes[1].set_title('Total Profit by Region')
axes[1].set_xlabel('Region')
axes[1].set_ylabel('Total Profit')
axes[1].ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.show()

# Observe: Are there regional disparities in sales and profit? This helps target marketing.

# 6. Sales and Profit by Customer Segment
fig, axes = plt.subplots(2, 1, figsize=(10, 12))

# Sales by Segment
sales_by_segment = df.groupby('Segment')['Sales'].sum().sort_values(ascending=False)
sns.barplot(x=sales_by_segment.index, y=sales_by_segment.values, ax=axes[0], palette='cubehelix')
axes[0].set_title('Total Sales by Customer Segment')
axes[0].set_xlabel('Segment')
axes[0].set_ylabel('Total Sales')
axes[0].ticklabel_format(style='plain', axis='y')

# Profit by Segment
profit_by_segment = df.groupby('Segment')['Profit'].sum().sort_values(ascending=False)
sns.barplot(x=profit_by_segment.index, y=profit_by_segment.values, ax=axes[1], palette='rocket')
axes[1].set_title('Total Profit by Customer Segment')
axes[1].set_xlabel('Segment')
axes[1].set_ylabel('Total Profit')
axes[1].ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.show()

# Observe: Which segments are most profitable? How do sales and profit differ across segments?

# 7. Time-Series Analysis: Sales and Profit Over Time
# Resample data to monthly frequency
monthly_sales_profit = df.set_index('Order Date')[['Sales', 'Profit']].resample('M').sum()

plt.figure(figsize=(14, 6))
plt.plot(monthly_sales_profit.index, monthly_sales_profit['Sales'], label='Monthly Sales', marker='o', markersize=4)
plt.plot(monthly_sales_profit.index, monthly_sales_profit['Profit'], label='Monthly Profit', marker='x', markersize=4)
plt.title('Monthly Sales and Profit Trends Over Time')
plt.xlabel('Date')
plt.ylabel('Amount')
plt.legend()
plt.grid(True)
plt.ticklabel_format(style='plain', axis='y')
plt.show()

# Observe: Identify overall trends (growth/decline), seasonality (e.g., end-of-year spikes),
# and any unusual dips or peaks.

# Sales and Profit by Year
yearly_sales_profit = df.groupby('Order Year')[['Sales', 'Profit']].sum()

plt.figure(figsize=(12, 6))
yearly_sales_profit.plot(kind='bar', y=['Sales', 'Profit'], figsize=(10, 6))
plt.title('Yearly Sales and Profit')
plt.xlabel('Year')
plt.ylabel('Amount')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.ticklabel_format(style='plain', axis='y')
plt.show()

# Sales and Profit by Month (aggregated across all years to see seasonality)
monthly_avg_sales_profit = df.groupby('Order Month')[['Sales', 'Profit']].mean()

plt.figure(figsize=(12, 6))
monthly_avg_sales_profit.plot(kind='bar', y=['Sales', 'Profit'], figsize=(10, 6))
plt.title('Average Monthly Sales and Profit (Aggregated)')
plt.xlabel('Month')
plt.ylabel('Average Amount')
plt.xticks(ticks=range(12), labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], rotation=45)
plt.grid(axis='y')
plt.ticklabel_format(style='plain', axis='y')
plt.show()

# Observe: Clear seasonality? Which months are strongest/weakest?

# 8. Quantity, Discount, and their impact on Sales/Profit
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(x='Quantity', y='Profit', data=df, alpha=0.6)
plt.title('Quantity vs. Profit')
plt.xlabel('Quantity')
plt.ylabel('Profit')

plt.subplot(1, 2, 2)
sns.scatterplot(x='Discount', y='Profit', data=df, alpha=0.6)
plt.title('Discount vs. Profit')
plt.xlabel('Discount')
plt.ylabel('Profit')
plt.tight_layout()
plt.show()

# Observe: Discounts often lead to negative profits. This is a critical insight.

# Correlation Matrix for numerical features
# Ensure 'Shipping Cost' and 'Discount' are present in the new dataset as well
# The US Superstore Data.csv typically has these.
numerical_cols = ['Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost', 'Profit Ratio', 'DaysToShip']
# Filter out columns that might not exist in the specific loaded dataset (e.g., if 'Shipping Cost' was missing)
actual_numerical_cols = [col for col in numerical_cols if col in df.columns]
plt.figure(figsize=(10, 8))
sns.heatmap(df[actual_numerical_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

print("EDA phase completed. Insights have been generated.")